# Import

In [77]:
import pandas as pd
import re

In [78]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [79]:
train_df = pd.read_csv("../../data/train_df_translate_del.csv")
test_df = pd.read_csv("../../data/test_df_translate_del.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [80]:
print(len(train_df))
print(len(test_df))
print(len(submission))

54609
23405
23405


제목과 키워드 열을 기준으로 중복된 행 제거

In [81]:
train_df.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

In [82]:
print(len(train_df))
print(len(test_df))
print(len(submission))

54315
23405
23405


In [83]:
import pandas as pd

# NaN 값을 찾는 함수
def find_nan_values(df, column_name):
    nan_rows = df[df[column_name].isna()]
    return nan_rows

# train_df와 test_df에서 '키워드' 열의 NaN 값 찾기
nan_train = find_nan_values(train_df, '키워드')
nan_test = find_nan_values(test_df, '키워드')

print("Train DataFrame에서 NaN 값이 있는 행들:")
print(nan_train)

print("\nTest DataFrame에서 NaN 값이 있는 행들:")
print(nan_test)

Train DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 분류, 제목, 키워드]
Index: []

Test DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 제목, 키워드]
Index: []


## 전처리

영어로만 구성된 단어에 대한 처리 (주소 제거)

In [84]:
import pandas as pd
import re

# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_specific_words)
test_df['키워드'] = test_df['키워드'].apply(remove_specific_words)

In [85]:
print(len(test_df))
print(len(submission))

23405
23405


In [86]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
train_english_words = []
test_english_words = []

train_df['키워드'].apply(lambda x: train_english_words.extend(extract_english_keywords(x)))
test_df['키워드'].apply(lambda x: test_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_english_word_counts = pd.Series(train_english_words).value_counts()
test_english_word_counts = pd.Series(test_english_words).value_counts()

# 5개 이하로 등장하는 단어 제거
train_words_to_remove = train_english_word_counts[train_english_word_counts <= 5].index.tolist()
test_words_to_remove = test_english_word_counts[test_english_word_counts <= 5].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_english_word_counts = train_english_word_counts[train_english_word_counts > 5]
filtered_test_english_word_counts = test_english_word_counts[test_english_word_counts > 5]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_english_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_english_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
AI       3602
SK       3307
GTX      2951
LH       2919
IC       1840
SNS      1642
CCTV     1526
GH       1508
KLPGA    1407
TF       1384
dtype: int64

Test DataFrame 상위 10개의 결과:
AI       1575
SK       1358
GTX      1275
LH       1255
GH        836
IC        776
SNS       717
TF        619
ESG       613
KLPGA     579
dtype: int64


In [87]:
print(len(test_df))
print(len(submission))

23405
23405


영어, 한글 혼합 단어 처리

In [88]:
import pandas as pd
import re

# 제거할 단어 패턴 리스트
remove_patterns = [
    r'http', r'www\.', r'\.kr', r'\.net', r'\.com',
    r'[A-Z]씨', r'[A-Z]양', r'[A-Z]군', r'[A-Z]사', r'[A-Z]팀', r'[A-Z]업체', r'[A-Z]노선',
    r'[A-Z]등급', r'[A-Z]자형', r'[A-Z]의원', r'[A-Z]그룹', r'[A-Z]타입', r'[A-Z]교사', r'[A-Z]법인',
    r'[A-Z]선수', r'[A-Z]시행사', r'[A-Z]대표', r'[A-Z]경위', r'[A-Z]시의원', r'[A-Z]센터', r'[A-Z]교회',
    r'[A-Z]아파트', r'[A-Z]학생', r'[A-Z]값', r'[A-Z]공인중개사사무소', r'[A-Z]캐피탈', r'[A-Z]고등학교',
    r'[A-Z]경사', r'[A-Z]농장', r'[A-Z]지구', r'[A-Z]구역', r'[A-Z]산후조리원', r'[A-Z]노래방', r'[A-Z]사령관',
    r'[A-Z]동', r'[A-Z]원장', r'[A-Z]카페', r'[A-Z]연구소', r'[A-Z]주식회사', r'[A-Z]요양병원', r'[A-Z]채널',
    r'[A-Z]연습장', r'[A-Z]학원', r'[A-Z]건설사', r'[A-Z]초등학교', r'[A-Z]시장', r'[A-Z]식품접객업소',
    r'[A-Z]한약국', r'[A-Z]한약방', r'[A-Z]모사', r'[A-Z]회사', r'[A-Z]경찰관', r'[A-Z]기업', r'[A-Z]업체',
    r'[A-Z]공인중개사사무소', r'[A-Z]도시', r'[A-Z]회장', r'[A-Z]동물병원', r'[A-Z]요양원', r'[A-Z]도의원',
    r'[A-Z]변호사', r'[A-Z]마트', r'[A-Z]병원', r'[A-Z]공인중개사', r'[A-Z]협회', r'[A-Z]사장', r'[A-Z]개발',
    r'[A-Z]조합', r'[A-Z]노선', r'[A-Z]구', r'[A-Z]그룹', r'[A-Z]자', r'[A-Z]공', r'[A-Z]검사', r'[A-Z]팀',
    r'[A-Z]중학교', r'[A-Z]건설사', r'[A-Z]컨벤션', r'[A-Z]건설', r'[A-Z]농협', r'[A-Z]약국', r'[A-Z]구역',
    r'[A-Z]은행', r'[A-Z]헬스장', r'[A-Z]팀', r'[A-Z]예식장', r'[A-Z]단지', r'[A-Z]지점', r'[A-Z]지자체',
    r'[A-Z]교회', r'[A-Z]블록', r'[A-Z]목사', r'[A-Z]교사', r'[A-Z]골프장', r'[A-Z]건설', r'[A-Z]종합부동산금융사',
    r'[A-Z]세무서장', r'[A-Z]산업', r'[A-Z]골프장', r'[A-Z]강사', r'[A-Z]원장', r'[A-Z]기업', r'[A-Z]타워',
    r'[A-Z]현장', r'[A-Z]소방관', r'[A-Z]동', r'[A-Z]양', r'[A-Z]초등학생', r'[A-Z]언론사', r'[A-Z]당원',
    r'[A-Z]시', r'[A-Z]상병', r'[A-Z]교수', r'[A-Z]학원', r'[A-Z]아파트', r'[A-Z]등', r'[A-Z]구'
]

# 정규표현식을 사용하여 단어 제거 함수
def remove_exact_words(text):
    for pattern in remove_patterns:
        text = re.sub(pattern, '', text)
    return text

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_exact_words)
test_df['키워드'] = test_df['키워드'].apply(remove_exact_words)

In [89]:
# import pandas as pd

# # 제거할 단어 리스트
# remove_words = [
#     'http', 'www.', '.kr', '.net', '.com',
#     'A씨', 'B씨', 'A양', 'C씨', 'C양', 'D씨', 'A군', 'A사', 'B군', 'B사', 'F팀', 'A업체', 'C노선',
#     'B양', 'D등급', 'L자형', 'A의원', 'E씨', 'A그룹', 'A타입', 'C등급', 'B그룹', 'A교사', 'C법인',
#     'A선수', 'A법인', 'B등급', 'B노선', 'B타입', 'J씨', 'A시행사', 'A대표', 'G사', 'E등급', 'A경위',
#     'B법인', 'R구역', 'A시의원', 'P사', 'K씨', 'C그룹', 'D센터', 'B업체', 'A교회', 'D사', 'H사', 'A아파트',
#     'B학생', 'C값', 'F씨', 'S씨', 'C군', 'H공인중개사사무소', 'A캐피탈', 'S등급', 'A고등학교', 'A경사', 'B교회',
#     'D법인', 'C사', 'G씨', 'A골프장', 'F노선', 'D노선', 'm가량', 'A팀장', 'L씨', '플랜B', 'B의원', 'A어린이집',
#     'S사', 'A농장', 'B지구', 'D구역', 'A산후조리원', 'L사', 'A노래방', 'A사령관', 'Y씨', 'B교사', 'A동', 'C의원',
#     'A원장', 'A카페', 'A연구소', 'E주식회사', 'B아파트', 'B어린이집', 'B경장', 'A요양병원', 'F채널', 'A학생',
#     'A연습장', 't가량', 'A학원', 'A건설사', 'B동', 'B초등학교', 'B건설사', 'B시장', 'n차', 'p차', 'A식품접객업소',
#     'C법', 'B한약국', 'C한약방', 'E군', 't트럭', 'G모사', 'A회사', 'A경찰관', 'A기업', 'B학원', 'C업체',
#     'A공인중개사사무소', 'A도시', 'A회장', 'V그룹', 'A고교', 'B동물병원', 'A요양원', 'SA등급', 'B도의원',
#     'C변호사', 'B시 B씨네', 'A마트', 'C병원', 'A병원', 'D아파트', 'B공인중개사', 'B선수', 'A협회', 'A사장',
#     'D양', 'B개발', 'A조합', 'E노선', '원x2.5', 'A구', 'G그룹', 'S자', 'A공', 'A검사', 'n곤충.kr', 'A팀', '0반',
#     'C중학교', 'P차이', 'km가량', 'B조', 'J건설사', 'C컨벤션', 'D건설', 'SS이슈', '시즌A', '시즌B', '시즌C', 'A농협',
#     'A약국', 'C약국', 'A구역', '존x', 'A은행', 'kw급', 'D군', 'A헬스장', 'B팀', 'C예식장', 'A단지', 'A지점', 'A지자체',
#     't급', 'M교회', 'C블록', 'B목사', '플랜C', 'B번', '개XX', 'D교사', 'B골프장', 'H건설', 'A종합부동산금융사',
#     'B세무서장', 'H산업', 'T골프장', 'H씨', 'A강사', 'B강사', 'B원장', 'J기업', 'M타워', 'A현장', 'A소방관', 'C동',
#     'F양', 'A초등학생', 'A언론사', 'A당원', 'A시', 'C상병', 'C교수', 'C학원', 'C아파트', 'A등', 'X새끼', 'D구'
# ]

# def remove_exact_words(text):
#     words = text.split(',')
#     filtered_words = [word for word in words if word not in remove_words]
#     return ','.join(filtered_words)

# # '키워드' 열에 함수 적용
# train_df['키워드'] = train_df['키워드'].apply(remove_exact_words)
# test_df['키워드'] = test_df['키워드'].apply(remove_exact_words)

In [90]:
print(len(test_df))
print(len(submission))

23405
23405


In [91]:
import pandas as pd
import re

# 영어와 한글이 혼합된 단어를 필터링하는 함수
def is_mixed_korean_english(word):
    return bool(re.search('[a-zA-Z]', word) and re.search('[가-힣]', word))

# 영어와 한글이 혼합된 단어를 추출하는 함수
def extract_mixed_keywords(keywords):
    words = keywords.split(',')
    mixed_words = [word for word in words if is_mixed_korean_english(word)]
    return mixed_words

# 모든 영어와 한글이 혼합된 단어를 하나의 리스트로 모으기
train_mixed_words = []
test_mixed_words = []

train_df['키워드'].apply(lambda x: train_mixed_words.extend(extract_mixed_keywords(x)))
test_df['키워드'].apply(lambda x: test_mixed_words.extend(extract_mixed_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_mixed_word_counts = pd.Series(train_mixed_words).value_counts()
test_mixed_word_counts = pd.Series(test_mixed_words).value_counts()

# 3개 이하로 등장하는 단어 제거
train_words_to_remove = train_mixed_word_counts[train_mixed_word_counts <= 3].index.tolist()
test_words_to_remove = test_mixed_word_counts[test_mixed_word_counts <= 3].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_mixed_word_counts = train_mixed_word_counts[train_mixed_word_counts > 3]
filtered_test_mixed_word_counts = test_mixed_word_counts[test_mixed_word_counts > 3]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_mixed_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_mixed_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
SK하이닉스      4040
DL이앤씨        323
CJ대한통운       273
QR코드         269
D2블록         224
HL디앤아이한라     214
LG전자         197
삼성SDI        191
SK에코플랜트      191
부동산R114      168
dtype: int64

Test DataFrame 상위 10개의 결과:
SK하이닉스       1548
DL이앤씨         133
CJ대한통운        127
QR코드          114
D2블록           86
HL디앤아이한라       82
SK에코플랜트        81
HDC현대산업개발      79
부동산R114        77
LG전자           72
dtype: int64


In [92]:
print(len(test_df))
print(len(submission))

23405
23405


기호 제거

In [93]:
df_combined = pd.concat([train_df, test_df], axis=0)

In [94]:
# 기호를 추출하는 함수
def extract_symbols(text):
    return re.findall(r'[^\w\s,]', text)  # 알파벳, 숫자, 공백, 쉼표를 제외한 모든 기호

# '키워드' 열에서 기호 추출
symbols = df_combined['키워드'].apply(extract_symbols)

# 기호 리스트 출력
unique_symbols = set([symbol for sublist in symbols for symbol in sublist])
print(unique_symbols)

{'→', '“', ';', '‧', '○', '▲', '▷', '*', '⋯', '㈜', '!', '.', '(', ')', '․', '㎡', '~', '%', '?', '-', '˙', '⋅', '·', '■', '’', '&', '・', '∙', '/', '•', ':'}


In [95]:
import pandas as pd

# 기호 집합 정의
symbols_to_remove = {
    '∙', '㈜', '→', ')', ':', '/', '•', '⋯', ';', '“', '・'
    , '(', '.', '~', '■', '‧', '*', '?', '▷', '㎡', '⋅'
    , '․', '○', '!', '=', '·', '˙', '’', '%', '▲'
    # , '&', '-'
}

# 키워드에서 기호가 포함된 단어를 제거하는 함수
def remove_keywords_with_symbols(keywords):
    filtered_keywords = []
    for keyword in keywords.split(','):
        if not any(symbol in keyword for symbol in symbols_to_remove):
            filtered_keywords.append(keyword)
    return ','.join(filtered_keywords)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_keywords_with_symbols)
test_df['키워드'] = test_df['키워드'].apply(remove_keywords_with_symbols)

# 결과 확인
print("Train DataFrame '키워드' 열:")
print(train_df['키워드'].head())

print("\nTest DataFrame '키워드' 열:")
print(test_df['키워드'].head())

Train DataFrame '키워드' 열:
0    용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...
1    용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트...
2    용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화...
3    수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특...
4    용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코...
Name: 키워드, dtype: object

Test DataFrame '키워드' 열:
0    김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대...
1    신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바...
2    전해철,장관,재정,분권,강화,지방자치,시대,마중물,마련,장관,전해철,행정안전부,풀뿌...
3    용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의...
4    지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경...
Name: 키워드, dtype: object


'경기용'과 '전문' 단어 제거

In [96]:
# '경기용'과 '전문' 단어 제거 함수
def remove_keywords(keywords, words_to_remove):
    keywords_list = keywords.split(', ')
    filtered_keywords = [word for word in keywords_list if word not in words_to_remove]
    return ', '.join(filtered_keywords)

# 제거할 단어 목록
words_to_remove = ['경기용', '전문']

# '키워드' 열 업데이트
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))

ᅳᆯ 있는 행 제거

In [97]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'ᅳᆯ', na=False)]

# 결과 출력
filtered_df.iloc[0]['키워드']

'김동성,양육비,배드파더스,등재,평생,꼬리표,인민정,호소,전문,김동성,쇼트,트랙,국가,대표,극단적,선택,인민정,여자,친구,인민정,자신,SNS,심경,인스타그램,저,ᆼ글맘,자신,처지,공개,우울증,공황,장애,김동성,공평,상황들,뇌리,장문,김동성,9ᄎ,ᅥᆫ만원정도,살아야,로,고,그,꼬리표는,평생,달고,사,토로,김동성씨,방송,ᅳᆯ,도,아무것,ᅡᄀ,스,써,여,ᆷᄒ,ᅩᄅ,데,극단적,ᅳᆯ,할,수밖에,어,바닥,어,이,지옥,상황,결과,호소,김동성,지난달,용인시,상현동,자택,극단적,선택,시도,신고,출동,119구급,대원,병원,이송,생명,지장,ᅦᄀ,ᅳᆫ,질타,않으면,ᆯᄋ,ᅳᆫ데,저는,제가,눈을,뜨고,있는,한,어떠한,상황에서도,발벗고,나서지,아,일분이,가,이,이,이,소중하기ᄋ,ᅳᆫ,야,설명,배드파더스,경제활동,악의적,양육비,비양육자,채찍질,소득,수입,재산,비양육자,배드파더스,낙인,합리적,개인적,생각,아내,이혼,양육비,논란,김동성,TV조선,이혼,프로그램,출연,인민정,인스타,그램,전문,싱글맘,양육자와,비양육자,ᆼ글맘입니다,지금껏,양육비ᄂ,이엄마입니다,도,ᅳᆯ,걸까,이,이,서,이,벌었습니다,도,아,ᅳᆯ,ᅳᆯ,김동성씨ᄂ,고,랴,로,이,9ᄎ,ᅥᆫ만원정도입니다,도,로,고,그,꼬리표는,평생,달고,사,남자친구,ᅦᄀ,ᅳᆫ,고,느,ᆯᄋ,ᅳᆫ데,저는,제가,눈을,뜨고,있는한,어떠한,상황에서도,발벗고,나서지,않으면,지금,우리는,올ᄉ,ᆼ입니다,가,이,이,이,ᅳᆫ,없습니다,도,가야합ᄂ,에,

In [98]:
# 해당 행 제거
train_df = train_df.drop(filtered_df.index)

In [99]:
print(len(test_df))
print(len(submission))

23405
23405


In [100]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자

        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+'                          # 숫자
    )
    valid_keywords = []
    exceptions = ['코로나19', 'RE100', 'GS25']
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            if word not in exceptions:  # 예외 처리
                removed_keywords.append(word)
            else:
                valid_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


제거된 단어들:
['18일', '15일', '7월', '8월', '22일', '20명', '10명', '70명', '36종류', '10시']


소분류 기준으로 제거

In [101]:
# 많이 등장한 공통 키워드 추출 함수
def filter_keywords(df, min_count=10, common_threshold=50):
    '''
    min_count: 최소 등장 횟수
    common_threshold: 공통 키워드의 최소 빈도수
    ex) min_count=10, common_threshold=50 -> 50개 이상의 분류에서 10번 이상 등장한 키워드만 선택
    '''
    # 키워드 분리 및 집계
    df['키워드'] = df['키워드'].str.split(',')
    df_exploded = df.explode('키워드')  # 각 리스트의 요소를 개별 행으로 나눔

    # 분류별 키워드 집계
    keyword_counts = df_exploded.groupby(['분류', '키워드']).size().reset_index(name='빈도수')

    # n개 이상 등장한 키워드 추출
    filtered_keywords = keyword_counts[keyword_counts['빈도수'] >= min_count]

    # 공통 키워드 찾기
    keyword_occurrences = filtered_keywords['키워드'].value_counts()
    common_keywords = keyword_occurrences[keyword_occurrences >= common_threshold].index.tolist()

    # 삭제되는 키워드와 개수
    num_deleted_keywords = len(common_keywords)

    # 삭제되는 키워드의 총 등장 횟수 계산
    total_deleted_occurrences = 0
    for kw in common_keywords:
        total_deleted_occurrences += keyword_occurrences.get(kw, 0)

    # 결과 출력
    print("삭제되는 키워드:", common_keywords)
    print("삭제되는 키워드의 개수(unique): {}".format(num_deleted_keywords))
    print("삭제되는 키워드의 총 개수: {}".format(total_deleted_occurrences))

In [102]:
df4 = train_df.copy()
filter_keywords(df4, 15, 45)

삭제되는 키워드: [' 용인시', ' 경기', ' 서울', ' 경기도', ' 상황', ' 진행', ' 대표', ' 용인', ' 이날', ' 지역', ' 시작', ' 예정', ' 관계자', ' 생각', ' 사진', ' 설명', ' 참여', ' 시장', ' 코로나19', ' 활동', ' 마련', ' 지원', ' 전국', ' 운영', ' 시민', ' 제공', ' 한국', ' 시민들', ' 준비', ' 모습', ' 가능', ' 노력', ' 사람', ' 계획', ' 정도', ' 자리', ' 기록', ' 확인', ' 대상', ' 위치']
삭제되는 키워드의 개수(unique): 40
삭제되는 키워드의 총 개수: 1915


In [103]:
# 공통 키워드를 제거하는 함수
def remove_common_keywords(df, common_keywords):
    for index, row in df.iterrows():
        keywords = row['키워드'].split(',')
        # 공통 키워드를 제외한 새로운 키워드 리스트
        filtered_keywords = [kw for kw in keywords if kw not in common_keywords]
        # 필터링된 키워드를 문자열로 다시 합침
        df.at[index, '키워드'] = ', '.join(filtered_keywords)
    return df

In [104]:
common_keywords_list = [' 용인시', ' 경기', ' 서울', ' 경기도', ' 상황', ' 진행', ' 대표', ' 용인'
                        , ' 이날', ' 지역', ' 시작', ' 예정', ' 관계자', ' 생각', ' 사진', ' 설명'
                        , ' 참여', ' 시장', ' 코로나19', ' 활동', ' 마련', ' 지원', ' 전국', ' 운영'
                        , ' 시민', ' 제공', ' 한국', ' 시민들', ' 준비', ' 모습', ' 가능', ' 노력'
                        , ' 사람', ' 계획', ' 정도', ' 자리', ' 기록', ' 확인', ' 대상', ' 위치']

train_df = remove_common_keywords(train_df, common_keywords_list)
test_df = remove_common_keywords(test_df, common_keywords_list)

In [105]:
train_df['키워드']

0        용인문화재단,  인문학,  콘서트,  뮤지컬,  이야기,  개최,  인문학,  콘서...
1        용인,  농촌,  테마파크,  단체,  체험객,  체험료,  체험일,  기준,  용...
2        용인시,  노후,  주택,  에너지,  성능,  개선,  신청,  연장,  용인시청...
3        수원,  고양시,  특례시,  지정,  도시경쟁력,  증가,  도내,  인구,  수...
4        용인시,  스페인,  미국,  국제,  명예,  자문관,  위촉,  역할,  대외,...
                               ...                        
54604    용인,  아파트,  여성,  아들,  추락,  극단,  선택,  추정,  경찰,  ...
54605    용인시,  위탁,  부모,  보수,  교육,  남부,  가정,  위탁,  센터,  ...
54606    용인시,  플랫폼,  시티,  국토부,  신청,  사업,  인정,  협의,  중앙토...
54607    주민자치위원회,  이동읍,  주민,  자치,  위원회,  가구,  밑반찬,  처인구...
54608    용인시,  용인시공무원노조,  국무총리,  수상,  홍화표,  용인시공무원노동조합,...
Name: 키워드, Length: 54314, dtype: object

In [106]:
# 키워드를 정리하는 함수
def clean_keywords(keywords):
    # 쉼표로 분리한 후 양쪽 공백 제거 및 빈 키워드 제외
    cleaned_keywords = [keyword.strip() for keyword in keywords.split(',') if keyword.strip()]
    # 공백으로 결합하여 하나의 문자열로 만듦
    return ' '.join(cleaned_keywords)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(clean_keywords)
test_df['키워드'] = test_df['키워드'].apply(clean_keywords)

In [107]:
train_df['키워드']

0        용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화...
1        용인 농촌 테마파크 단체 체험객 체험료 체험일 기준 용인시통합예약사이트 접수 용인특...
2        용인시 노후 주택 에너지 성능 개선 신청 연장 용인시청 노후 건축물 환경친화적 개선...
3        수원 고양시 특례시 지정 도시경쟁력 증가 도내 인구 수원 고양 특례시 지정 골자 지...
4        용인시 스페인 미국 국제 명예 자문관 위촉 역할 대외 홍보 권태면 주코스타리카 대사...
                               ...                        
54604    용인 아파트 여성 아들 추락 극단 선택 추정 경찰 현장 유서 경위 조사 처인구 아파...
54605    용인시 위탁 부모 보수 교육 남부 가정 위탁 센터 시청 컨퍼런스 위탁 부모 보수 교...
54606    용인시 플랫폼 시티 국토부 신청 사업 인정 협의 중앙토지수용위원회 국토 교통부 중앙...
54607    주민자치위원회 이동읍 주민 자치 위원회 가구 밑반찬 처인구 이동읍 주민 자치 위원회...
54608    용인시 용인시공무원노조 국무총리 수상 홍화표 용인시공무원노동조합 공무원 노사 문화 ...
Name: 키워드, Length: 54314, dtype: object

In [126]:
train_df_ex = train_df.copy()

In [128]:
def get_length(text):
    return len(text.split())

# 제목, 키워드, 전체 텍스트의 길이 계산
train_df_ex['키워드_length'] = train_df_ex['키워드'].apply(get_length)

# 길이 통계 출력
print("\n키워드 길이 통계:")
print(train_df_ex['키워드_length'].describe())


키워드 길이 통계:
count    54314.000000
mean       142.450786
std        101.524451
min          3.000000
25%         83.000000
50%        120.000000
75%        176.000000
max       2456.000000
Name: 키워드_length, dtype: float64


90% 지점인 200정도가 max length 적절해 보임

In [129]:
print(len(test_df))
print(len(submission))

23405
23405


분류의 갯수 확인

In [137]:
train_df['분류'].value_counts(0).head(4)

지역          26850
경제:부동산       3447
사회:사건_사고     2545
경제:반도체       2309
Name: 분류, dtype: int64

In [138]:
train_df['분류'].value_counts(1).head(4)

지역          0.494348
경제:부동산      0.063464
사회:사건_사고    0.046857
경제:반도체      0.042512
Name: 분류, dtype: float64

분류 : 지역 이 절반정도 차지하고 있음

In [139]:
import os
import pandas as pd
from datetime import datetime

# 현재 날짜 가져오기 (MMDD 형식)
current_date = datetime.now().strftime('%m%d')

# CSV 파일명 생성
train_csv_filename = f"train_df_{current_date}.csv"
test_csv_filename = f"test_df_{current_date}.csv"

# CSV 파일로 저장
train_df.to_csv(train_csv_filename, index=False, encoding='utf-8-sig')
test_df.to_csv(test_csv_filename, index=False, encoding='utf-8-sig')

print(f"train_df saved as {train_csv_filename}")
print(f"test_df saved as {test_csv_filename}")

train_df saved as train_df_1007.csv
test_df saved as test_df_1007.csv


.